In [ ]:
!pip install pymysql
!pip install skyfield

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import pymysql
from sqlalchemy import create_engine
from skyfield.api import load
from datetime import datetime, timedelta
import pytz
import pandas as pd
import requests
from http.server import BaseHTTPRequestHandler, HTTPServer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
import json

# Параметры подключения к базе данных
db_config = {
    "host": "sql7.freemysqlhosting.net",
    "user": "sql7753415",
    "password": "XNjmZyasHS",
    "database": "sql7753415",
    "port": 3306
}

DB_SERVER = "sql7.freemysqlhosting.net"
DB_NAME = "sql7753415"
DB_USER = "sql7753415"
DB_PASSWORD = "XNjmZyasHS"
DB_PORT = 3306



BASE_URL = "https://iss.moex.com/iss/history/engines/stock/markets/shares/securities"

def fetch_first_trading_day(company):

    try:
        response = requests.get(f"{BASE_URL}/{company}.json", params={"iss.meta": "off", "lang": "ru"})
        response.raise_for_status()

        # Парсинг данных
        data = response.json()
        history_data = data.get('history', {}).get('data', [])
        history_columns = data.get('history', {}).get('columns', [])

        # Преобразование в DataFrame
        df = pd.DataFrame(history_data, columns=history_columns)

        # Проверка и нахождение первой даты
        if "TRADEDATE" in df.columns:
            return df["TRADEDATE"].min()
        else:
            raise ValueError("TRADEDATE колонка не найдена в данных.")
    except Exception as e:
        raise RuntimeError(f"Ошибка при запросе первого дня торгов: {e}")

def fetch_historical_data(company, start_date):

    all_data = []
    start = 0
    try:
        while True:
            # Параметры запроса
            params = {
                "from": start_date,
                "iss.meta": "off",
                "start": start,  # Пагинация
                "lang": "ru"
            }

            response = requests.get(f"{BASE_URL}/{company}.json", params=params)
            response.raise_for_status()

            # Парсинг данных
            data = response.json()
            history_data = data.get('history', {}).get('data', [])
            history_columns = data.get('history', {}).get('columns', [])

            if not history_data:
                break  # Если данные закончились

            df = pd.DataFrame(history_data, columns=history_columns)
            all_data.append(df)

            # Увеличение пагинации
            start += 100

        # Объединение всех данных
        full_data = pd.concat(all_data, ignore_index=True)

        # Фильтрация данных
        if "TRADEDATE" in full_data.columns and "CLOSE" in full_data.columns:
            result_df = full_data[["TRADEDATE", "CLOSE"]].rename(columns={
                "TRADEDATE": "date",
                "CLOSE": "value"
            })
            return result_df
        else:
            raise ValueError("Не найдены необходимые колонки (TRADEDATE и CLOSE).")
    except Exception as e:
        raise RuntimeError(f"Ошибка при запросе исторических данных: {e}")

def company_stock_history(company):

    try:
        first_day = fetch_first_trading_day(company)
        historical_data = fetch_historical_data(company, first_day)
        return historical_data
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def merge_stock_with_astro(stock_data):

    if 'date' not in stock_data.columns:
        raise ValueError("Датасет 'stock_data' должен содержать столбец 'date'.")

    # Подключение к базе данных
    engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}")

    # Определяем диапазон дат из stock_data
    start_date = stock_data['date'].min()
    end_date = stock_data['date'].max()

    # SQL-запрос для извлечения данных из таблицы astro в указанном диапазоне
    query = f"""
    SELECT *,
           CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) AS full_date
    FROM astro
    WHERE CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) BETWEEN '{start_date}' AND '{end_date}';
    """

    # Извлечение данных из базы данных
    astro_data = pd.read_sql(query, engine)

    # Приводим столбец full_date в astro_data к формату datetime
    astro_data['full_date'] = pd.to_datetime(astro_data['full_date'])

    # Приводим столбец date в stock_data к формату datetime
    stock_data['date'] = pd.to_datetime(stock_data['date'])

    # Объединяем stock_data с astro_data по столбцу "date" и "full_date"
    eph_stock_data = pd.merge(stock_data, astro_data, left_on='date', right_on='full_date', how='left')

    # Убираем вспомогательный столбец full_date
    eph_stock_data = eph_stock_data.drop(columns=['full_date'])

    return eph_stock_data

In [ ]:
def prepare_and_train_model_lstm(eph_stock_data, target_column='value', timesteps=30, epochs=20, batch_size=32):
    # 1. Удаляем строки с NaN
    eph_stock_data = eph_stock_data.dropna()

    # 2. Удаляем ненужные столбцы
    eph_stock_data = eph_stock_data.drop(columns=['date_x', 'date_y'], errors='ignore')

    # 3. Масштабируем данные к диапазону [-1, 1], кроме целевого столбца
    scaler = StandardScaler()
    feature_columns = eph_stock_data.columns.difference([target_column])
    eph_stock_data[feature_columns] = scaler.fit_transform(eph_stock_data[feature_columns])

    # 4. Подготавливаем данные для LSTM
    X, y = [], []
    for i in range(len(eph_stock_data) - timesteps):
        X.append(eph_stock_data.iloc[i:i+timesteps][feature_columns].values)
        y.append(eph_stock_data.iloc[i+timesteps][target_column])

    X = np.array(X)
    y = np.array(y)

    # 5. Создаем LSTM модель
    input_shape = (X.shape[1], X.shape[2])  # (timesteps, features)
    model = Sequential()
    model.add(LSTM(64, activation='tanh', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))  # Предсказание одного значения
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

    # 6. Обучаем модель
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True)
    callbacks = [early_stopping, model_checkpoint]

    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=callbacks, verbose=1)

    return model, scaler

def get_stock_predict_lstm(trained_model, scaler, stock_data, target_column='value', timesteps=30):
    # 1. Подготовка данных эфемерид
    engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}")
    moscow_timezone = pytz.timezone("Europe/Moscow")
    current_time = datetime.now(moscow_timezone)

    start_date = stock_data['date'].max() + pd.Timedelta("1 day")
    end_date = stock_data['date'].max() + pd.Timedelta("31 day")

    query = f"""
    SELECT *,
           CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) AS full_date
    FROM astro
    WHERE CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) BETWEEN '{start_date}' AND '{end_date}';
    """
    astro_data = pd.read_sql(query, engine)
    astro_data['date'] = pd.to_datetime(astro_data[['year', 'month', 'day']])

    # 2. Объединяем данные
    stock_data['date'] = pd.to_datetime(stock_data['date'])
    merged_data = pd.merge(stock_data, astro_data, on='date', how='right')
    merged_data = merged_data.drop(columns=['date', 'full_date', 'value'], errors='ignore')

    feature_columns = merged_data.columns.difference([target_column])
    merged_data[feature_columns] = scaler.transform(merged_data[feature_columns])

    # 3. Подготовка временного окна для предсказания
    initial_window = merged_data.iloc[-timesteps:][feature_columns].values
    X_pred = np.array([initial_window])

    # 4. Генерация предсказаний на 30 дней
    predictions = []
    for _ in range(30):
        pred_value = trained_model.predict(X_pred)[0][0]
        predictions.append(pred_value)
        new_row = np.zeros((1, X_pred.shape[2]))
        X_pred = np.concatenate([X_pred[:, 1:, :], new_row[np.newaxis, :, :]], axis=1)
        X_pred[0, -1, :-1] = merged_data.iloc[-1][feature_columns]
        X_pred[0, -1, -1] = pred_value

    future_dates = [start_date + pd.Timedelta(f"{i} day") for i in range(30)]
    prediction_df = pd.DataFrame({'date': future_dates, 'predicted_value': predictions})

    return prediction_df

def calculate(tiker):

    stock_data = company_stock_history(tiker)
    eph_stock_data = merge_stock_with_astro(stock_data)
    trained_model, scaler = prepare_and_train_model_lstm(eph_stock_data)
    predicted_prices = get_stock_predict_lstm(trained_model,scaler, stock_data)

    return stock_data, predicted_prices

def convert_dates_to_strings(df):

    if 'date' in df.columns:
        df['date'] = df['date'].astype(str)
    return df

In [ ]:
df1, df2 = calculate("YDEX")
print(df2)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 259ms/step - loss: 14687503.0000 - mae: 3825.1250 - val_loss: 11761997.0000 - val_mae: 3424.4031
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 14648432.0000 - mae: 3820.0137 - val_loss: 11760277.0000 - val_mae: 3424.1489
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 14630211.0000 - mae: 3817.3826 - val_loss: 11758767.0000 - val_mae: 3423.9250
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 14541310.0000 - mae: 3805.8865 - val_loss: 11757532.0000 - val_mae: 3423.7407
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 14662460.0000 - mae: 3821.7427 - val_loss: 11755954.0000 - val_mae: 3423.5056
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 14700806.0000 - mae: 3826.5916 - val_loss: 11753741.0000 - val_mae: 3423.1765
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 14589060.0000 - mae: 3812.2188 - val_loss: 11752125.0000 - val_mae: 3422.9329
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 14636860

ValueError: could not broadcast input array from shape (27,) into shape (26,)